## Build kerchunk_mapping_ooiDAS script

In [16]:
import fsspec
import kerchunk.hdf
import zarr
import h5py

import xarray as xr
import dask
from dask.distributed import Client
import os
import sys
from azure.storage.blob import BlobServiceClient, ContainerClient
from tqdm import tqdm
import ujson
import multiprocessing

In [40]:
# Set the account name, container name, and file path
account_name = "dasdata"
container_name = "hdf5"
connection_string = os.environ['AZURE_CONSTR_dasdata']

file_path = "South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T231114Z.h5"

In [4]:
# Create a container client object
container_client = ContainerClient.from_connection_string(conn_str=connection_string, container_name=container_name)

# Create an empty list to hold file names
file_names = []

# Loop through all blobs in the container and append their names to the list
for blob in container_client.list_blobs():
    file_names.append(blob.name)

In [20]:
len(file_names)

3981

In [ ]:
def get_kerchunk_mapping(file):
    print(multiprocessing.current_process().name)
    
    # Set the blob URL using the fsspec AzureBlobFileSystem
    url = f"az://{container_name}/{file}"
    openfile = fsspec.open(url, account_name=account_name)
    
    # Generate dictionary
    h5chunks = kerchunk.hdf.SingleHdf5ToZarr(openfile.open(), url)

    outf = f'json_files/{file[:-3]}.json'
    with fs2.open(outf, 'wb') as f:
        f.write(ujson.dumps(h5chunks.translate()).encode())

# Create a multiprocessing pool with the number of available CPUs
pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())

# Map the file processing function to the pool
results = pool.map(get_kerchunk_mapping, file_names)

# Close the pool to release resources
pool.close()
pool.join()

In [ ]:
fs2 = fsspec.filesystem('')  #local file system to save final jsons to

outs = []
for file in tqdm(file_names):

    # Set the blob URL using the fsspec AzureBlobFileSystem
    url = f"az://{container_name}/{file}"
    openfile = fsspec.open(url, account_name=account_name)

    # Generate dictionary
    h5chunks = kerchunk.hdf.SingleHdf5ToZarr(openfile.open(), url)
    
    #out.append(h5chunks.translate())
    
    outf = f'json_files/{file[:-3]}.json'
    with fs2.open(outf, 'wb') as f:
        f.write(ujson.dumps(h5chunks.translate()).encode());

## Combine Files

In [49]:
from kerchunk.combine import MultiZarrToZarr
import glob

In [70]:
# TODO remove absolute path
json_file_path = '/home/jhrag/Code/ODLintake/scripts/json_files/'
json_list = [f'{json_file_path}{f}' for f in os.listdir(json_file_path)]

In [71]:
for f in json_list:
    if f[-4:] != 'json':
        json_list.remove(f)
        print(f'removed {f}')

removed /home/jhrag/Code/ODLintake/scripts/json_files/.ipynb_checkpoints


In [73]:
mzz = MultiZarrToZarr(
    json_list,
    remote_protocol="az",
    storage_options=dict(account_name=account_name, connection_string=connection_string),
    concat_dims=["time"]
)

In [74]:
mzz.translate()

JSONDecodeError: Expected object or value